ATIVIDADE PROPOSTA: Você trabalha em uma empresa de varejo e precisa analisar os dados de vendas do último ano
para identificar padrões e insights para melhorar o desempenho. Os dados estão armazenados
em um banco de dados SQLite, e você utilizará a biblioteca Pandas para manipular e analisar
esses dados, além de gerar visualizações utilizando Matplotlib e Seaborn.

In [ ]:
# Passo 1: Conectar ao banco de dados SQLite e criar uma tabela
import sqlite3
conn = sqlite3.connect("dados_vendas.db")
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS vendas1")
cursor.execute ("""
     CREATE TABLE IF NOT EXISTS vendas1(
       id_venda INTEGER PRIMARY KEY AUTOINCREMENT,
       data_venda DATE,
       produto TEXT,
       categoria TEXT,
       valor_venda REAL
     )
""")
conn.commit()


In [ ]:
import pandas as pd
# Define a formatação para todos os números flutuantes
# A formatação R$ {:.2f} força duas casas decimais
pd.options.display.float_format = 'R$ {:.2f}'.format

# Passo 2: Inserir alguns dados
cursor.execute('''
INSERT INTO vendas1 (data_venda, produto, categoria, valor_venda) VALUES
('2023-01-01', 'Produto A', 'Eletrônicos', 1500.00),
('2023-01-05', 'Produto B', 'Roupas', 350.00),
('2023-02-10', 'Produto C', 'Eletrônicos', 1200.00),
('2023-03-15', 'Produto D', 'Livros', 200.00),
('2023-03-20', 'Produto E', 'Eletrônicos', 800.00),
('2023-04-02', 'Produto F', 'Roupas', 400.00),
('2023-05-05', 'Produto G', 'Livros', 150.00),
('2023-06-10', 'Produto H', 'Eletrônicos', 1000.00),
('2023-07-20', 'Produto I', 'Roupas', 600.00),
('2023-08-25', 'Produto J', 'Eletrônicos', 700.00),
('2023-09-30', 'Produto K', 'Livros', 300.00),
('2023-10-05', 'Produto L', 'Roupas', 450.00),
('2023-11-15', 'Produto M', 'Eletrônicos', 900.00),
('2023-12-20', 'Produto N', 'Livros', 250.00);
''')


conn.commit()


In [ ]:
# Passo 2: Extrair os dados do banco de dados para um DataFrame do Pandas
df = pd.read_sql_query("SELECT * FROM vendas1", conn)


In [ ]:
# Deletando um produto (através do seu id)
cursor.execute("DELETE FROM vendas1 WHERE id_venda = ?", (10,))
conn.commit()
#conn.close()

In [ ]:
# Atualizar o preço de um produto
cursor.execute("UPDATE vendas1 SET valor_venda = 950.00 WHERE id_venda = 13")
conn.commit()

In [ ]:
# Inserindo um novo produto na tabela
novo_produto = ("2024-03-25", "Produto O", "Livros", 225.00)
cursor.execute("INSERT INTO vendas1(data_venda, produto, categoria, valor_venda)"
"VALUES(?, ?, ?, ?)", novo_produto)
conn.commit()


In [ ]:
# Passo 4: Recarregar o DataFrame com os dados atualizados
df_atualizado = pd.read_sql_query("SELECT * FROM vendas1", conn)
conn.close()

In [ ]:
# Calcula o total de vendas por data
vendas_por_data = df_atualizado.groupby('data_venda')['valor_venda'].sum()

In [ ]:
# Encontra a data e o valor de maior faturamento
data_maior_faturamento = vendas_por_data.idxmax()
valor_maior_faturamento = vendas_por_data.max()

In [ ]:
# Encontra a data e o valor de menor faturamento
data_menor_faturamento = vendas_por_data.idxmin()
valor_menor_faturamento = vendas_por_data.min()


In [ ]:
# Passo 5: Exibir os resultados
# Passo 5.1: Tabela 'Vendas do Produto'

print("                --- Tabela Produtos ---")
print(df)


In [ ]:
# Passo 5.2: Tabela 'Venda dos Produtos'(Atualizada)
print("\n            --- Tabela Produtos(Atualizada) ---")
print(df_atualizado)
print("")

In [ ]:
# Valor total das vendas (calculado com a tabela ATUALIZADA)
valor_total_vendas = df_atualizado.valor_venda.sum()
#print("\nValor total das vendas:")
print(f"Valor total das vendas:R$ {valor_total_vendas:.2f}")


Valor total das vendas:R$ 8375.00


In [ ]:
# Vendas por categoria (calculado com a tabela ATUALIZADA)
vendas_por_categoria = df_atualizado.groupby('categoria')['valor_venda'].sum().reset_index()
print("\nValor total vendido por categoria:")
print(vendas_por_categoria.to_string())

In [ ]:
# Data e valor do dia de maior venda
print("\nData de Maior Faturamento:")
print(f"Data: {data_maior_faturamento}")
print(f"Valor total: R$ {valor_maior_faturamento:.2f}")

In [ ]:
# Data e valor da venda do dia de menor venda
print("\nData de Menor Faturamento:")
print(f"Data: {data_menor_faturamento}")
print(f"Valor total: R$ {valor_menor_faturamento:.2f}")


Data de Menor Faturamento:
Data: 2023-05-05
Valor total: R$ 150.00


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Passo 8: Criar o Gráfico de Barras
plt.figure(figsize=(10, 6))
sns.barplot(x='categoria', y='valor_venda', data=vendas_por_categoria, palette='viridis', hue='categoria', legend=False)
plt.title('Total de Vendas por Categoria', fontsize=16)
plt.xlabel('Categoria', fontsize=12)
plt.ylabel('Valor Total de Vendas (R$)', fontsize=12)
plt.show()


In [ ]:
# Cria o gráfico de linhas
plt.figure(figsize=(12, 6))
vendas_por_data.plot(kind='line', marker='o')

# Adiciona títulos e rótulos
plt.title('Evolução das Vendas ao Longo do Tempo', fontsize=16)
plt.xlabel('Data da Venda', fontsize=12)
plt.ylabel('Valor Total de Vendas (R$)', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
# Prepara os dados: a variável vendas_por_categoria já está disponível
# O Matplotlib precisa dos valores e dos rótulos separadamente
valores = vendas_por_categoria['valor_venda']
rotulos = vendas_por_categoria['categoria']

# Cria o gráfico de pizza
plt.figure(figsize=(8, 8)) # Define o tamanho do gráfico
plt.pie(valores, labels=rotulos, autopct='%1.1f%%', startangle=90, colors=plt.cm.Paired.colors)

# Adiciona um título
plt.title('Participação de Vendas por Categoria', fontsize=16)

# Garante que o gráfico de pizza seja um círculo
plt.axis('equal')
plt.show()